In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/:D

/content/gdrive/MyDrive/:D


In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from PIL import Image


tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.17.0


In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
input=np.load("input.npy")
target=np.load("target.npy")

In [ ]:
input=np.delete(input, 2, axis=3)

In [ ]:
# Set the seed for reproducibility
np.random.seed(42)

# Generate shuffled indices for the first dimension
shuffled_indices = np.random.permutation(input.shape[0])

# Apply the shuffled indices to the first dimension of both arrays
input = input[shuffled_indices, :, :, :]
target = target[shuffled_indices, :, :, :]

In [ ]:
#20 80 split
input, input_evaluation, target, target_evaluation = train_test_split(input, target, test_size=0.2, random_state=42)

PRE-PROCESSING - NEW FEATURES

MODEL ARCHITECTURE

In [ ]:
#DICE METRIC TO USE FOR TRAINING

import tensorflow as tf
from keras.metrics import Metric

class CustomDiceMetric(Metric):
    def __init__(self, penalization_coefficients=[1.0, 1.0, 1.0], smooth=1e-6, **kwargs):
        super(CustomDiceMetric, self).__init__(**kwargs)
        self.penalization_coefficients = penalization_coefficients
        self.smooth = smooth
        self.total_dice = self.add_weight(name='total_dice', initializer='zeros')
        self.num_samples = self.add_weight(name='num_samples', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.reshape(y_true, (-1, 256 * 304, 3))
        y_pred = tf.reshape(y_pred, (-1, 256 * 304, 3))

        dice_value = 0.0
        for i in range(3):  # Assuming 3 channels
            y_true_channel = y_true[:, :, i]
            y_pred_channel = y_pred[:, :, i]

            intersection = tf.reduce_sum(y_true_channel * y_pred_channel, axis=-1)
            union = tf.reduce_sum(y_true_channel, axis=-1) + tf.reduce_sum(y_pred_channel, axis=-1)

            dice = (2.0 * intersection + self.smooth) / (union + self.smooth)
            dice_value += dice * self.penalization_coefficients[i]

        self.total_dice.assign_add(tf.reduce_sum(dice_value))
        self.num_samples.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.total_dice / self.num_samples

    def reset_states(self):
        # Reset the state of the metric
        self.total_dice.assign(0.0)
        self.num_samples.assign(0.0)

# Example usage:
dice_metric = CustomDiceMetric(penalization_coefficients=[0.1, 0.45, 0.45], smooth=1e-6)

In [ ]:
#CUSTOM METRICS TO EVALUATE MODEL AFTER TRAINING


import tensorflow.keras.backend as K

def precision(y_true, y_pred, name):
  y_true=K.round(y_true)
  y_pred=K.round(y_pred)

  if(name == 'background'):
    channel = 0
  elif(name == 'nanobars'):
    channel = 1
  elif(name == 'nanoparticles'):
    channel = 2
  else:
    raise ValueError("Invalid channel name, strunz")

  true_positives = K.sum(K.round(K.clip(y_true[:,:,:,channel] * y_pred[:,:,:,channel], 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred[:,:,:,channel], 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  return precision


def recall(y_true, y_pred, name):

  y_true=K.round(y_true)
  y_pred=K.round(y_pred)

  if(name == 'background'):
    channel = 0
  elif(name == 'nanobars'):
    channel = 1
  elif(name == 'nanoparticles'):
    channel = 2
  else:
    raise ValueError("Invalid channel name, strunz")


  true_positives = K.sum(K.round(K.clip(y_true[:,:,:,channel] * y_pred[:,:,:,channel], 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true[:,:,:,channel], 0, 1)))
  recall = true_positives / (possible_positives + K.epsilon())
  return recall


def f1_score(y_true, y_pred, name):
    y_true=K.round(y_true)
    y_pred=K.round(y_pred)

    if(name == 'background'):
      channel = 0
    elif(name == 'nanobars'):
      channel = 1
    elif(name == 'nanoparticles'):
      channel = 2
    else:
      raise ValueError("Invalid channel name, strunz")

    p = precision(y_true, y_pred, name)
    r = recall(y_true, y_pred, name)
    f1_score = 2 * ((p * r) / (p + r + K.epsilon()))
    return f1_score


def custom_accuracy(y_true, y_pred, name):
    if name == 'background':
        channel = 0
    elif name == 'nanobars':
        channel = 1
    elif name == 'nanoparticles':
        channel = 2

    # Assuming y_true and y_pred are 4D tensors with the last dimension representing channels
    # Flatten the tensors to compare pixel-wise accuracy
    y_true_flat = tf.reshape(y_true[:, :, :, channel], (-1,))
    y_pred_flat = tf.reshape(y_pred[:, :, :, channel], (-1,))

    # Calculate pixel-wise accuracy
    correct_pixels = tf.reduce_all(tf.equal(tf.round(y_true_flat), tf.round(y_pred_flat)), axis=-1)
    accuracy = tf.reduce_mean(tf.cast(correct_pixels, tf.float32))

    return accuracy

In [ ]:
input_shape= (256,304,2)
output_shape= (256,304,3)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose , UpSampling2D , Concatenate , Cropping2D , BatchNormalization , Dropout


num_filtros = [32,64,80]
dropout = [0, 0.2 , 0.4]
inception = [False, True]



def build_model(num_filtros,dropout):

    '''Build the neural network layer by layer'''

    # LAYER 1 DOWN

    input_layer = tfkl.Input(shape=input_shape, name='Input')
    conv_1_1 = tfkl.Conv2D(filters= num_filtros,
                           kernel_size=(3,3),
                           strides=(1,1),
                           padding='same',
                           activation='relu',
                           )(input_layer)
    drop_1 = Dropout(dropout, seed=seed) (conv_1_1)
    conv_1_2 = tfkl.Conv2D(filters= num_filtros,
                           kernel_size=(3,3),
                           strides=(1,1),
                           padding='same',
                           activation='relu')(drop_1)
    drop_2 = Dropout(dropout, seed=seed) (conv_1_2)



    # LAYER 2 DOWN

    pool_2_1 = tfkl.MaxPooling2D(pool_size=(2,2))(drop_2)
    conv_2_2 = tfkl.Conv2D(filters=num_filtros*2 ,
                           kernel_size=(3,3),
                           strides=(1,1),
                           padding='same',
                           activation='relu')(pool_2_1)
    drop_3 = Dropout(dropout, seed=seed) (conv_2_2)
    conv_2_3 = tfkl.Conv2D(filters= num_filtros*2,
                           kernel_size=(3,3),
                           strides=(1,1),
                           padding='same',
                           activation='relu')(drop_3)
    drop_4 = Dropout(dropout, seed=seed) (conv_2_3)



    # LAYER 3 DOWN

    pool_3_1 = tfkl.MaxPooling2D(pool_size=(2,2))(drop_4)
    conv_3_2 = tfkl.Conv2D(filters= num_filtros*4,
                           kernel_size=(3,3),
                           strides=(1,1),
                           padding='same',
                           activation='relu')(pool_3_1)
    drop_5 = Dropout(dropout, seed=seed) (conv_3_2)
    conv_3_3 = tfkl.Conv2D(filters= num_filtros*4 ,
                           kernel_size=(3,3),
                           strides=(1,1),
                           padding='same',
                           activation='relu')(drop_5)
    drop_6 = Dropout(dropout, seed=seed) (conv_3_3)



    # LAYER 4 DOWN

    pool_4_1 = tfkl.MaxPooling2D(pool_size=(2,2))(drop_6)
    conv_4_2 = tfkl.Conv2D(filters= num_filtros*8,
                           kernel_size=(3,3),
                           strides=(1,1),
                           padding='same',
                           activation='relu')(pool_4_1)
    drop_7 = Dropout(dropout, seed=seed) (conv_4_2)
    conv_4_3 = tfkl.Conv2D(filters= num_filtros*8,
                           kernel_size=(3,3),
                           strides=(1,1),
                           padding='same',
                           activation='relu')(drop_7)
    drop_8 = Dropout(dropout, seed=seed) (conv_4_3)



    # LAYER 5 DOWN

    pool_5_1 = tfkl.MaxPooling2D(pool_size=(2,2))(drop_8)
    conv_5_2 = tfkl.Conv2D(filters=num_filtros*16 ,
                           kernel_size=(3,3),
                           strides=(1,1),
                           padding='same',
                           activation='relu')(pool_5_1)
    drop_9 = Dropout(dropout, seed=seed) (conv_5_2)
    conv_5_3 = tfkl.Conv2D(filters= num_filtros*16,
                           kernel_size=(3,3),
                           strides=(1,1),
                           padding='same',
                           activation='relu')(drop_9)
    drop_10 = Dropout(dropout, seed=seed) (conv_5_3)


    # LAYER 1 UP

    up_conv_6_1 = tfkl.Conv2DTranspose(filters= num_filtros*8,
                                       kernel_size=(3,3),
                                       strides=(2,2),
                                       padding='same',
                                       activation='relu')(drop_10)
    conc_6_2 = tfkl.Concatenate(axis=-1)([conv_4_3,up_conv_6_1])
    conv_6_3 = tfkl.Conv2D(filters= num_filtros*8,
                           kernel_size=(3,3),
                           strides=(1,1),
                           padding='same',
                           activation='relu')(conc_6_2)
    drop_11 = Dropout(dropout, seed=seed) (conv_6_3)
    conv_6_4 = tfkl.Conv2D(filters= num_filtros*8,
                           kernel_size=(3,3),
                           strides=(1,1),
                           padding='same',
                           activation='relu')(drop_11)
    drop_12 = Dropout(dropout, seed=seed) (conv_6_4)

    # LAYER 2 UP

    up_conv_7_1 = tfkl.Conv2DTranspose(filters= num_filtros*4,
                                       kernel_size=(3,3),
                                       strides=(2,2),
                                       padding='same',
                                       activation='relu')(drop_12)
    conc_7_2 = tfkl.Concatenate(axis=-1)([conv_3_3,up_conv_7_1])
    conv_7_3 = tfkl.Conv2D(filters= num_filtros*4,
                           kernel_size=(3,3),
                           strides=(1,1),
                           padding='same',
                           activation='relu')(conc_7_2)
    drop_13 = Dropout(dropout, seed=seed) (conv_7_3)
    conv_7_4 = tfkl.Conv2D(filters= num_filtros*4,
                           kernel_size=(3,3),
                           strides=(1,1),
                           padding='same',
                           activation='relu')(drop_13)
    drop_14 = Dropout(dropout, seed=seed) (conv_7_4)

    # LAYER 3 UP

    up_conv_8_1 = tfkl.Conv2DTranspose(filters= num_filtros*2,
                                       kernel_size=(3,3),
                                       strides=(2,2),
                                       padding='same',
                                       activation='relu')(drop_14)
    conc_8_2 = tfkl.Concatenate(axis=-1)([conv_2_3,up_conv_8_1])
    conv_8_3 = tfkl.Conv2D(filters= num_filtros*2,
                           kernel_size=(3,3),
                           strides=(1,1),
                           padding='same',
                           activation='relu')(conc_8_2)
    drop_15 = Dropout(dropout, seed=seed) (conv_8_3)
    conv_8_4 = tfkl.Conv2D(filters= num_filtros*2 ,
                           kernel_size=(3,3),
                           strides=(1,1),
                           padding='same',
                           activation='relu')(drop_15)
    drop_16 = Dropout(dropout, seed=seed) (conv_8_4)

    # LAYER 4 UP

    up_conv_9_1 = tfkl.Conv2DTranspose(filters=num_filtros,
                                       kernel_size=(3,3),
                                       strides=(2,2),
                                       padding='same',
                                       activation='relu')(drop_16)
    conc_9_2 = tfkl.Concatenate(axis=-1)([conv_1_2,up_conv_9_1])
    conv_9_3 = tfkl.Conv2D(filters=num_filtros,
                           kernel_size=(3,3),
                           strides=(1,1),
                           padding='same',
                           activation='relu')(conc_9_2)
    drop_17 = Dropout(dropout, seed=seed) (conv_9_3)
    conv_9_4 = tfkl.Conv2D(filters=num_filtros,
                           kernel_size=(3,3),
                           strides=(1,1),
                           padding='same',
                           activation='relu')(drop_17)
    drop_18 = Dropout(dropout, seed=seed) (conv_9_4)
    conv_9_5 = tfkl.Conv2D(filters=num_filtros,
                           kernel_size=(3,3),
                           strides=(1,1),
                           padding='same',
                           activation='relu')(drop_18)
    drop_19 = Dropout(dropout, seed=seed) (conv_9_5)


    # OUTPUT LAYER


    output_layer_1 = tfkl.Conv2D(num_filtros,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(drop_19)
    output_layer_2 = tfkl.Conv2D(int(num_filtros/2) ,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(output_layer_1)
    output_layer_3 = tfkl.Conv2D(int(num_filtros/4) ,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(output_layer_2)
    output_layer_4 = tfkl.Conv2D(int(num_filtros/8) ,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(output_layer_3)
    output_layer = tfkl.Conv2D(3,kernel_size=(3,3),strides=(1,1),padding='same',activation='sigmoid')(output_layer_4)
    #resizing_layer = tfkl.Resizing(height=885, width=1024, interpolation='bicubic')(output_layer)





    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss= 'binary_crossentropy', optimizer=tfk.optimizers.Adam(), metrics=[dice_metric])

    # Return the model
    return model


model = build_model(32,0.2)



In [ ]:
batch_size=5
epochs=300

from re import M
# Train the model with cross-validation

names = ('background', 'nanobars', 'nanoparticles')
f1 = np.zeros(shape=(1, len(names)))
prec = np.zeros(shape=(1, len(names)))
rec = np.zeros(shape=(1, len(names)))


hitories = []

In [ ]:



for i in range(0,1):

  history = model.fit(x = input,
                    y = target,
                    batch_size = batch_size,
                    epochs = epochs,

                    shuffle = False,
                    ).history  # perform early stopping on the metric that we're interested in, i.e. DICE METRIC

  hitories.append(history)
  h = model.predict(input_evaluation)
  h = h.astype(np.float32)

  m=0
  for j in names:
    f1[i,m] = f1_score(target_evaluation, h, j)
    prec[i,m] = precision(target_evaluation, h, j)
    rec[i,m] = recall(target_evaluation, h, j)


    m+=1

  print(f1)
  print(rec)
  print(prec)





Epoch 1/300
57/57 ━━━━━━━━━━━━━━━━━━━━ 88s 636ms/step - custom_dice_metric: 0.1572 - loss: 0.6357
Epoch 2/300
57/57 ━━━━━━━━━━━━━━━━━━━━ 15s 167ms/step - custom_dice_metric: 0.4081 - loss: 0.1093
Epoch 3/300
57/57 ━━━━━━━━━━━━━━━━━━━━ 10s 168ms/step - custom_dice_metric: 0.4056 - loss: 0.1020
Epoch 4/300
57/57 ━━━━━━━━━━━━━━━━━━━━ 10s 167ms/step - custom_dice_metric: 0.4162 - loss: 0.0922
Epoch 5/300
57/57 ━━━━━━━━━━━━━━━━━━━━ 10s 166ms/step - custom_dice_metric: 0.4472 - loss: 0.0781
Epoch 6/300
57/57 ━━━━━━━━━━━━━━━━━━━━ 10s 167ms/step - custom_dice_metric: 0.4457 - loss: 0.0766
Epoch 7/300
57/57 ━━━━━━━━━━━━━━━━━━━━ 9s 163ms/step - custom_dice_metric: 0.4517 - loss: 0.0743
Epoch 8/300
57/57 ━━━━━━━━━━━━━━━━━━━━ 10s 164ms/step - custom_dice_metric: 0.4532 - loss: 0.0723
Epoch 9/300
57/57 ━━━━━━━━━━━━━━━━━━━━ 10s 165ms/step - custom_dice_metric: 0.4552 - loss: 0.0718
Epoch 10/300
57/57 ━━━━━━━━━━━━━━━━━━━━ 10s 165ms/step - custom_dice_metric: 0.4563 - loss: 0.0716
Epoch 11/300
57/57 ━

In [ ]:
model.save('2_32_02.keras')